In [4]:
import requests,json,pprint
import pandas as pd
import pprint
from api_keys import lda_senate_api_key

headers = {"Authorization": f"Token {lda_senate_api_key}"}

url = "https://lda.senate.gov/api/v1/contributions/?filing_year=2023&page=50"
response = requests.get(url,headers=headers).json()
#response = requests.get(url).json()
pprint.pprint(response)
##response

{'detail': 'Invalid page.'}


In [ ]:
# This section does a couple things:
# 1. Begins populating new_dict_list with information we want. Later it is converted to a df
# 2. Counts how many results exist and how many contributions exist in each result to make sense 
# of what is being pulled versus what is present. Essentially, to make sure all are accounted for.
# 3. It only loops through the first three pages as a test

new_dict_list = []

# Even though the URL is already defined above, if you don't define it again here,
# it will start at the last page left off as it keeps updating the global variable with the last page.
url = "https://lda.senate.gov/api/v1/contributions/?filing_year=2013&page=1"


# Reset cummulativeContributions
cummulativeContributions = 0

# Loop through first three pages
for i in range(1412):
    #url = f"https://lda.senate.gov/api/v1/contributions/?page={i}"
    print(url)
    response = requests.get(url,headers=headers).json()
    resultCount = 0
    totalContributions = 0
    for eachResult in response["results"]:
        contributionCount = 0
        for eachContribution in eachResult["contribution_items"]:
            
            # Store information from response as new entry in dictionary
            new_entry = {"date":eachContribution["date"],
                         "amount":eachContribution["amount"],
                         "honoree_name":eachContribution["honoree_name"],
                         "Payee Name":eachContribution["payee_name"],
                         "Contribution Type":eachContribution["contribution_type"],
                         "contributor_name":eachContribution["contributor_name"],
                         #"Lobbyist First Name":eachResult["lobbyist"]["first_name"],
                         #"Lobbyist Last Name":eachResult["lobbyist"]["last_name"],
                         #"Lobbyist Prefix":eachResult["lobbyist"]["prefix"],
                         "state":eachResult["state"],
                         "country":eachResult["country"],
                         "Registrant Name":eachResult["registrant"]["name"]}
            new_dict_list.append(new_entry)
            contributionCount =contributionCount+1
        #print(f"Contribution count is {contributionCount}")
        resultCount =resultCount+1
        totalContributions = totalContributions + contributionCount
    print(f"Result count is {resultCount}")
    print(f"Total Contributions for this page is {totalContributions}")
    cummulativeContributions = cummulativeContributions + totalContributions
    url = response["next"]
        
    
print(f"Cummulative contributions is {cummulativeContributions}")
df = pd.DataFrame(new_dict_list)

df.shape

In [ ]:
df

In [ ]:
df.to_csv("lda_senate_output/2013page1-1412.csv",index=False)